# 04_landing_move_files_to_bronze_job
---
Este notebook move os arquivos `parquet` da camada **Stage** para a camada **Bronze**, organizando-os por data de processamento.


In [45]:
# Parameters

stage_path = "/opt/airflow/data-layer/stage"
bronze_path = "/opt/airflow/data-layer/bronze"


In [44]:
import os
from datetime import datetime
from pyspark.sql import SparkSession
from transformer.utils.spark_helpers import get_spark_session
from transformer.utils.file_io import check_files_in_folder, move_files
from transformer.utils.logger import get_logger

log = get_logger("landing.move_to_bronze")

spark = get_spark_session("MoveStageToBronze")
log.info("[Landing][MoveToBronze] SparkSession iniciada.")


2025-11-12 00:04:34 [INFO] landing.move_to_bronze | [INFO] Logger inicializado no modo standalone (INFO).
2025-11-12 00:04:34 [INFO] spark_helpers | [INFO] SparkSession criada com sucesso: 'MoveStageToBronze' (master=local[*]).
2025-11-12 00:04:34 [INFO] landing.move_to_bronze | [Landing][MoveToBronze] SparkSession iniciada.


In [42]:
try:
    log.info("[Landing][MoveToBronze] Iniciando job de movimentação de arquivos.")

    parquet_files = check_files_in_folder(stage_path, "*.parquet")
    if not parquet_files:
        raise FileNotFoundError(f"[Landing][MoveToBronze][ERROR] Nenhum arquivo Parquet encontrado em {stage_path}.")

    processing_date = datetime.now().strftime("%Y-%m-%d")

    move_files(
        spark=spark,
        source_files=parquet_files,
        base_dest_path=bronze_path,
        processing_date=processing_date,
    )

    log.info(f"[Landing][MoveToBronze] {len(parquet_files)} arquivo(s) movido(s) para bronze/{processing_date}.")
    
    log.info("[Landing][MoveToBronze] Job concluído com sucesso.")

except Exception as e:
    log.exception(f"[Landing][MoveToBronze][ERROR] Falha durante execução: {e}")
    raise


2025-11-12 00:04:06 [INFO] landing.move_to_bronze | [Landing][MoveToBronze] Iniciando job de movimentação de arquivos.
2025-11-12 00:04:06 [INFO] file_io | [INFO] Encontrados 3 arquivo(s).
2025-11-12 00:04:06 [INFO] file_io | [INFO] Movendo arquivos para '/opt/airflow/data-layer/bronze'.
2025-11-12 00:04:06 [INFO] file_io | [INFO] Diretório criado: /opt/airflow/data-layer/bronze/2025-11-12/PARQUET
2025-11-12 00:04:06 [INFO] file_io | [INFO] 'airlines.parquet' movido para '/opt/airflow/data-layer/bronze/2025-11-12/PARQUET/airlines.parquet'.
2025-11-12 00:04:06 [INFO] file_io | [INFO] 'airports.parquet' movido para '/opt/airflow/data-layer/bronze/2025-11-12/PARQUET/airports.parquet'.
2025-11-12 00:04:06 [INFO] file_io | [INFO] 'flights.parquet' movido para '/opt/airflow/data-layer/bronze/2025-11-12/PARQUET/flights.parquet'.
2025-11-12 00:04:06 [INFO] file_io | [INFO] Movimentação concluída com sucesso.
2025-11-12 00:04:06 [INFO] landing.move_to_bronze | [Landing][MoveToBronze] 3 arquivo(

In [51]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.
path = "/opt/airflow/data-layer/bronze/2025-11-12/PARQUET/flights.parquet"

df = spark.read.parquet(str(path))
df.select("ORIGIN_AIRPORT").show(5, truncate=False)

In [50]:
# Encerrando a sessão Spark
spark.stop()
log.info("[Landing][MoveToBronze] Sessão Spark finalizada.")


2025-11-12 00:05:20 [INFO] landing.move_to_bronze | [Landing][MoveToBronze] Sessão Spark finalizada.
